In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [35]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51271_2024-05-02~2025-05-02 (1).xlsx')
na_df = pd.read_csv('/Users/seanyu_mac/Downloads/네이버스마트스토어 공구 - 시트1.csv')

In [37]:
fs_df = fs_df[fs_df['상품 번호'] == 51271]
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)
fs_df = fs_df[['결제 번호', '주문 상태','결제 완료일', '옵션명', '수량', '박스수량', '정산가']]

In [39]:
fs_df['구분'] = '친한스토어'

In [41]:
na_df['박스수량'] = na_df['옵션정보'].apply(lambda x: x.split('박스')[0].split('선택: ')[1])
na_df['박스수량'] = na_df['박스수량'].astype(int)
na_df['정산가'] = 0
con1 = na_df['박스수량'] == 3
con2 = na_df['박스수량'] == 5
con3 = na_df['박스수량'] == 8
na_df.loc[con1, '정산가'] = na_df.loc[con1, '수량'] * 63000
na_df.loc[con2, '정산가'] = na_df.loc[con2, '수량'] * 99000
na_df.loc[con3, '정산가'] = na_df.loc[con3, '수량'] * 149000

na_df = na_df[['상품주문번호', '주문상태', '주문일시', '옵션정보', '수량', '박스수량', '정산가']]

In [43]:
fs_df.columns

Index(['결제 번호', '주문 상태', '결제 완료일', '옵션명', '수량', '박스수량', '정산가', '구분'], dtype='object')

In [45]:
na_df.columns

Index(['상품주문번호', '주문상태', '주문일시', '옵션정보', '수량', '박스수량', '정산가'], dtype='object')

In [47]:
na_df['구분'] = '네이버스토어'
na_df.columns = fs_df.columns

In [49]:
temp_df = pd.concat([fs_df, na_df])

In [51]:
temp_df['주문 상태'].unique()

array(['결제완료', '취소완료', '배송완료'], dtype=object)

In [53]:
status = ['결제완료', '배송준비', '배송중', '배송완료']

In [55]:
temp_df[~(temp_df['주문 상태'].isin(status))]


,결제 번호,주문 상태,결제 완료일,옵션명,수량,박스수량,정산가,구분
1,parma20250429212411E9mZ1,취소완료,2025-04-29 21:25:32.107000,3박스,1,3,63000,친한스토어
14,parma20250425110502COti6,취소완료,2025-04-25 11:05:15.894000,3박스,1,3,63000,친한스토어


In [57]:
temp_df = temp_df[temp_df['주문 상태'].isin(status)]

In [59]:
temp_df['주문 상태'].unique()

array(['결제완료', '배송완료'], dtype=object)

In [61]:
temp_df['결제 완료일'] = pd.to_datetime(temp_df['결제 완료일'])
temp_df['결제 완료일'] = temp_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pivot = pd.pivot_table(temp_df, index=['결제 완료일', '옵션명'], values=['정산가', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합')
data = pd.pivot_table(temp_df, index=['결제 완료일', '옵션명'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')['결제 번호']
pivot['주문수량'] = data
pivot.columns = ['박스수량', '매출', '주문수량']
result = pd.DataFrame(pivot)
result = result[['매출', '주문수량', '박스수량']]
result.applymap(lambda x: f'{x:,}')

매출 주문수량 박스수량
결제 완료일     옵션명                         
2025-04-24 3박스        315,000    5   15
           5박스         99,000    1    5
           8박스        447,000    3   24
2025-04-25 3박스        189,000    3    9
           8박스        149,000    1    8
2025-04-26 5박스         99,000    1    5
           8박스        149,000    1    8
           선택: 8박스    149,000    1    8
2025-04-27 3박스        189,000    3    9
           8박스        149,000    1    8
           선택: 8박스    149,000    1    8
2025-04-28 3박스         63,000    1    3
           5박스         99,000    1    5
           선택: 5박스     99,000    1    5
2025-04-29 3박스         63,000    1    3
           5박스         99,000    1    5
총합                  2,506,000   26  128

In [63]:
pivot = pd.pivot_table(temp_df, index=['결제 완료일', '구분'], values=['정산가', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합')
data = pd.pivot_table(temp_df, index=['결제 완료일', '구분'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')['결제 번호']
pivot['주문수량'] = data
pivot.columns = ['박스수량', '매출', '주문수량']
result = pd.DataFrame(pivot)
result = result[['매출', '주문수량', '박스수량']]
result.applymap(lambda x: f'{x:,}')

매출 주문수량 박스수량
결제 완료일     구분                         
2025-04-24 친한스토어     861,000    9   44
2025-04-25 친한스토어     338,000    4   17
2025-04-26 네이버스토어    149,000    1    8
           친한스토어     248,000    2   13
2025-04-27 네이버스토어    149,000    1    8
           친한스토어     338,000    4   17
2025-04-28 네이버스토어     99,000    1    5
           친한스토어     162,000    2    8
2025-04-29 친한스토어     162,000    2    8
총합                 2,506,000   26  128